In [16]:
import os
from osgeo import gdal
import numpy as np
from PIL import Image
import xml.etree.ElementTree as ET

In [17]:
# Specify input and output folders
input_folder = "/Users/gilisaacgabriel/Library/CloudStorage/OneDrive-TheOhioStateUniversity/AppEARS DEM/NC"
output_folder = "/Users/gilisaacgabriel/Library/CloudStorage/OneDrive-TheOhioStateUniversity/AppEARS DEM/NC/PNG"


In [18]:
def get_statistics(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    statistics = {}

    metadata = root.find('.//Metadata')
    if metadata is not None:
        for mdi in metadata.findall('.//MDI'):
            statistics[mdi.attrib['key']] = float(mdi.text)

    return statistics

def normalize_array(array):
    """
    Normalize array values to range [0, 1]
    """
    min_val = np.min(array)
    max_val = np.max(array)
    normalized_array = (array - min_val) / (max_val - min_val)
    return normalized_array

def apply_color_mapping(array, min_val, max_val):
    """
    Apply color mapping based on minimum and maximum values
    """
    normalized_array = normalize_array(array)
    scaled_array = 255 * (normalized_array - min_val) / (max_val - min_val)
    return scaled_array.astype(np.uint8)

In [19]:
def stretch_contrast(image, min_val, max_val):
    """
    Stretch contrast of the image to the specified min and max values.
    """
    # Clip values below min_val and above max_val
    stretched_image = np.clip(image, min_val, max_val)
    
    # Normalize to range [0, 1]
    stretched_image = (stretched_image - min_val) / (max_val - min_val)
    
    # Scale to range [0, 255]
    stretched_image = (stretched_image * 255).astype(np.uint8)
    
    return stretched_image

def geotiff_to_png(input_folder, output_folder):
    # Make sure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get list of GeoTIFF files in the input folder
    tiff_files = [f for f in os.listdir(input_folder) if f.endswith('.tif') or f.endswith('.tiff')]

    for tiff_file in tiff_files:
        # Open GeoTIFF file
        tiff_path = os.path.join(input_folder, tiff_file)
        dataset = gdal.Open(tiff_path)
        if dataset is None:
            print(f"Could not open {tiff_file}")
            continue

        # Read raster data
        band = dataset.GetRasterBand(1)
        raster = band.ReadAsArray()

        # Get corresponding XML file
        xml_file = tiff_file + '.aux.xml'  # Adding .aux.xml suffix
        xml_path = os.path.join(input_folder, xml_file)

        # Check if XML file exists
        if not os.path.exists(xml_path):
            print(f"XML file not found for {tiff_file}")
            continue

        # Get statistics from XML file
        statistics = get_statistics(xml_path)

        # Extract min and max height from statistics
        min_height = statistics.get('STATISTICS_MINIMUM', 0)
        max_height = statistics.get('STATISTICS_MAXIMUM', 65535)  # Default to 65535 if maximum not found

        # Apply histogram stretching
        stretched_image = stretch_contrast(raster, min_height, max_height)

        # Create PNG image
        image = Image.fromarray(stretched_image)

        # Save PNG file with 16-bit depth
        png_file = os.path.splitext(tiff_file)[0] + '.png'
        png_path = os.path.join(output_folder, png_file)
        image.save(png_path, format='PNG', bitdepth=16)

        print(f"Converted {tiff_file} to {png_file}")

    print("Conversion complete.")

In [20]:
# Convert GeoTIFF files to PNG
geotiff_to_png(input_folder, output_folder)

Converted SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0009 (6).tif to SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0009 (6).png
Converted SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0024 (1).tif to SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0024 (1).png
Converted SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0023 (5).tif to SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0023 (5).png
Converted SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0066.tif to SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0066.png
Converted SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0072.tif to SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0072.png
Converted SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0020 (3).tif to SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0020 (3).png
Converted SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0021 (2).tif to SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0021 (2).png
Converted SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0073.tif to SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0073.png
Converted SRTMGL1_NC.003_SRTMGL1_DEM_doy2000042_aid0067.